<a href="https://colab.research.google.com/github/turkin501/CS116.M11.KHTN/blob/main/Ex_3/CS116_Ex3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install osmnet
!pip install matplotlib==3.1.3
!pip install osmnet
!pip install pandana
!pip install folium

!pip install rtree
!pip install pygeos
!pip install geojson
!pip install geopandas

     |████████████████████████████████| 994 kB 7.0 MB/s 
     |████████████████████████████████| 15.4 MB 38 kB/s 
     |████████████████████████████████| 6.3 MB 45.7 MB/s 
     |████████████████████████████████| 13.1 MB 99 kB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 994 kB 5.3 MB/s 
     |████████████████████████████████| 1.8 MB 5.3 MB/s 


In [ ]:
!git clone https://github.com/CityScope/CSL_HCMC.git

Cloning into 'CSL_HCMC'...
remote: Enumerating objects: 1930, done.
remote: Counting objects: 100% (590/590), done.
remote: Compressing objects: 100% (315/315), done.
remote: Total 1930 (delta 308), reused 500 (delta 269), pack-reused 1340
Receiving objects: 100% (1930/1930), 179.02 MiB | 24.27 MiB/s, done.
Resolving deltas: 100% (985/985), done.
Checking out files: 100% (542/542), done.


In [ ]:
path = "/content/CSL_HCMC/Data/GIS/Population/population_HCMC/population_shapefile/"

In [ ]:
import geopandas as gpd

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [ ]:
data = gpd.read_file(path + "Population_District_Level.shp")

data.head()

,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,geometry
0,District 1,760,District,180225.0,187435.0,187419.0,190943.0,197421.0,200297.0,193632.0,208475.0,211220.0,142625.0,23360.40,24294.90,24292.90,24749.60,25589.30,25962.10,25098.20,27022.1,27377.90,18486.8,15274.835115,7.714899e+06,"POLYGON ((684948.318 1193882.223, 684971.256 1..."
1,District 3,770,District,190553.0,188945.0,190631.0,191580.0,193206.0,193694.0,196333.0,198104.0,199095.0,190375.0,38688.60,38362.10,38704.40,38897.10,39227.20,39326.30,39862.10,40221.7,40422.90,38652.5,12182.058264,4.925215e+06,"POLYGON ((684382.122 1193521.050, 684410.753 1..."
2,District 4,773,District,180980.0,183261.0,184711.0,185649.0,185808.0,187157.0,186727.0,186995.0,183894.0,175329.0,43311.10,43857.00,44204.00,44428.40,44466.50,44789.30,44686.40,44750.6,44008.40,41958.7,9483.716891,4.178668e+06,"POLYGON ((687625.438 1189132.516, 687615.280 1..."
3,District 6,775,District,249329.0,253474.0,254212.0,257865.0,266121.0,257183.0,258945.0,260742.0,262757.0,233561.0,34910.50,35490.80,35594.20,36105.70,37261.60,36010.20,36256.90,36508.5,36790.60,32702.7,13077.365262,7.140945e+06,"POLYGON ((679534.413 1187518.498, 679077.904 1..."
4,District 7,778,District,244276.0,274828.0,268483.0,266633.0,280743.0,296757.0,310178.0,317488.0,327223.0,360155.0,6929.39,7796.06,7616.07,7563.59,7963.85,8418.12,8798.84,9006.2,9282.35,10216.5,33260.915360,3.525220e+07,"POLYGON ((691506.715 1190058.151, 691359.893 1..."


#Top 5 tăng dân số


In [ ]:
data['pop_spread'] = data['Pop_2019'] / data['Pop_2017']

top = data.sort_values(by='pop_spread', ascending=False).iloc[:5]
top

,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,geometry,pop_spread
16,District 9,763,District,256257.0,263486.0,271537.0,276432.0,284473.0,284990.0,290620.0,296198.0,310739.0,397006.0,2242.76,2306.02,2376.49,2419.33,2489.70,2494.23,2543.50,2592.32,2719.58,3474.59,53139.220230,1.142649e+08,"POLYGON ((701125.957 1205414.891, 701162.391 1...",1.277619
23,Hoc Mon,784,District,349065.0,358640.0,366502.0,396763.0,408683.0,414795.0,422471.0,434275.0,446056.0,542243.0,3197.13,3284.83,3356.84,3634.00,3743.18,3799.16,3869.46,3977.58,4085.48,4966.47,65955.793966,1.091744e+08,"POLYGON ((673061.206 1208548.077, 673094.550 1...",1.215639
18,Nha Be,786,District,101074.0,103793.0,110958.0,119416.0,126062.0,132034.0,139225.0,156001.0,177990.0,206837.0,1005.81,1032.87,1104.17,1188.34,1254.48,1313.90,1385.46,1552.41,1771.22,2058.29,67540.905029,1.004901e+08,"POLYGON ((688444.826 1185726.125, 688568.759 1...",1.162071
22,District 12,761,District,405360.0,427083.0,455881.0,475824.0,489511.0,499569.0,510326.0,526064.0,549064.0,620146.0,7680.92,8092.53,8638.21,9016.09,9275.44,9466.02,9669.85,9968.06,10403.90,11750.80,47703.394094,5.277342e+07,"POLYGON ((685183.891 1205265.848, 685198.395 1...",1.129460
17,District 2,769,District,147490.0,140621.0,137749.0,134131.0,140436.0,140288.0,147168.0,153832.0,161957.0,180275.0,2953.46,2815.91,2758.40,2685.95,2812.21,2809.25,2947.02,3080.46,3243.16,3609.98,39592.970451,4.993869e+07,"POLYGON ((690496.729 1195354.084, 690716.888 1...",1.113104


In [ ]:
my_file = "/content/drive/MyDrive/response.json"

clicks = gpd.read_file(my_file)

clicks.head()

,dbh,geometry
0,1,POINT (106.64816 10.78143)
1,1,POINT (106.69997 10.80689)
2,1,POINT (106.44963 10.99961)
3,1,POINT (106.70792 10.83483)
4,1,POINT (106.63170 10.82777)


In [ ]:
from geopandas.tools import sjoin

In [ ]:
top = top.to_crs('epsg:4326')

In [ ]:
click_w_district = gpd.sjoin(clicks, top, how ='left', op='within')

click_w_district.head()

,dbh,geometry,index_right,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,pop_spread
0,1,POINT (106.64816 10.78143),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,POINT (106.69997 10.80689),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,POINT (106.44963 10.99961),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,POINT (106.70792 10.83483),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,POINT (106.63170 10.82777),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
top_click = []

top_5_dist = list(set(top['Dist_Name']))

for i in range(5):
  coordinates = click_w_district[click_w_district['Dist_Name'] == top_5_dist[i]]
  top_click.append(coordinates)

In [ ]:
points_xy = []

for i in range(5):
    points_xy.append([[point.xy[1][0], point.xy[0][0]] for point in top_click[i].geometry]) 

#Kmeans

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

In [ ]:
list_pred = []
centers = []

for i in range(5):
  model = KMeans(n_clusters=30, random_state=0)
  y_pred = model.fit_predict(points_xy[i])
  centers.append(model.cluster_centers_)
  list_pred.append(y_pred)

In [ ]:
list_top = []

for i in range(5):
  label = list_pred[i]
  unique, counts = np.unique(label, return_counts=True)
  dict_value = dict(zip(unique, counts))
  inverse = [[value, key] for key, value in dict_value.items()]
  top = sorted(inverse, key=lambda k: [k[0]], reverse=True)[:1]
  top_labels = [el[1] for el in top]
  list_top.append(top_labels)

#Heatmap

In [ ]:
points_xy[0][0]

[10.8851935269015, 106.619319905948]

In [ ]:
import folium
from folium import plugins

map_top = folium.Map(location = points_xy[0][0], zoom_start=10)

colors = ["red","orange","yellow","green","blue"]

for i in range(5):
  point_plot = []
  for j in range(len(points_xy[i])):
    if list_pred[i][j] in list_top[i]:
      point_plot.append(points_xy[i][j])

  plugins.HeatMap(point_plot).add_to(map_top)

  center = centers[i][list_top[i]]
  folium.Marker(center[0], icon=folium.Icon(color=colors[i])).add_to(map_top)

map_top